In [93]:
import requests
from IPython.display import Markdown
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import altair as alt
from requests.utils import quote
import os
from github import Github
from datetime import timedelta

In [94]:
fmt = "{:%Y-%m-%d}"

# Can optionally use number of days to choose dates
n_days = 60
end_date = fmt.format(pd.datetime.today())
start_date = fmt.format(pd.datetime.today() - timedelta(days=n_days))

renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [95]:
# Calculate number of days to include in plots
n_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days

In [96]:
# Create the renderer we want
alt.renderers.enable(renderer);

In [97]:
# Define our query object that we'll re-use for github search
class GitHubGraphQlQuery():
    def __init__(self, query):
        self.query = query
        self.headers = {"Authorization": "Bearer %s" % os.environ['GITHUB_ACCESS_TOKEN']}
        self.gql_template = """
{
  search(%s) {
    issueCount
    pageInfo {
        endCursor
        hasNextPage
    }
    nodes {
      ... on PullRequest {
        state
        id
        title
        url
        createdAt
        updatedAt
        number
        authorAssociation
        author {
          login
          url
        }
        mergedBy {
          login
          url
        }
        comments(last: 15) {
          edges {
            node {
              authorAssociation
              createdAt
              lastEditedAt
              url
              repository {
                id
              }
              author {
                login
                url
              }
            }
          }
        }
      }
      ... on Issue {
        url
        title
        state
        createdAt
        updatedAt
        number
        authorAssociation
        author {
          login
          url
        }
        comments(last: 15) {
          edges {
            node {
              author {
                login
                url
              }
              authorAssociation
              createdAt
              lastEditedAt
              repository {
                id
              }
              url
            }
          }
        }
      }
    }
  }
}
"""
    def request(self, n_pages=100, n_per_page=50):
        self.raw_data = []
        for ii in range(n_pages):
            search_query = ["first: %s" % n_per_page, 'query: "%s"' % self.query, 'type: ISSUE']
            if ii != 0:
                search_query.append('after: "%s"' % pageInfo['endCursor'])

            this_query = self.gql_template % ', '.join(search_query)
            request = requests.post('https://api.github.com/graphql', json={'query': this_query}, headers=self.headers)
            if request.status_code != 200:
                raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, this_query))
            if "errors" in request.json().keys():
                raise Exception("Query failed to run with error {}. {}".format(request.json()['errors'], this_query))
            self.request = request

            # Parse the response
            json = request.json()['data']['search']
            if ii == 0:
                print("Found {} items, which will take {} pages".format(json['issueCount'], int(np.ceil(json['issueCount'] / n_per_page))))
            self.raw_data.append(json)
            pageInfo = json['pageInfo']
            self.last_query = this_query
            if pageInfo['hasNextPage'] is False:
                break
        
        if self.raw_data[0]['issueCount'] == 0:
            print("Found no entries for query {}".format(self.query))
            self.data = None
            return
        
        # Add some extra fields
        self.data = pd.DataFrame([jj for ii in self.raw_data for jj in ii['nodes']])
        self.data['username'] = self.data['author'].map(lambda a: a['login'])
        self.data['username_url'] = self.data['author'].map(lambda a: a['url'])
        self.data['org'] = self.data['url'].map(lambda a: a.split('/')[3])
        self.data['repo'] = self.data['url'].map(lambda a: a.split('/')[4])

In [98]:
# Discourse API key
api = {'Api-Key': os.environ['DISCOURSE_API_KEY'],
       'Api-Username': os.environ['DISCOURSE_API_USERNAME']}

In [99]:
# Discourse
def topics_to_markdown(topics, n_list=10):
    body = []
    for _, topic in topics.iterrows():
        title = topic['fancy_title']
        slug = topic['slug']
        posts_count = topic['posts_count']
        url = f'https://discourse.jupyter.org/t/{slug}'
        body.append(f'* [{title}]({url}) ({posts_count} posts)')
    body = body[:n_list]
    return '\n'.join(body)

def counts_from_activity(activity):
    counts = activity.groupby('category_id').count()['bookmarked'].reset_index()
    counts['parent_category'] = None
    for ii, irow in counts.iterrows():
        if parent_categories[irow['category_id']] is not None:
            counts.loc[ii, 'parent_category'] = parent_categories[irow['category_id']]

    counts['category_id'] = counts['category_id'].map(lambda a: category_mapping[a])
    counts['parent_category'] = counts['parent_category'].map(lambda a: category_mapping[a] if a is not None else 'parent')
    is_parent = counts['parent_category'] == 'parent'
    counts.loc[is_parent, 'parent_category'] = counts.loc[is_parent, 'category_id'] 
    counts['parent/category'] = counts.apply(lambda a: a['parent_category']+'/'+a['category_id'], axis=1)
    counts = counts.sort_values(['parent_category', 'bookmarked'], ascending=False)
    return counts

# Community forum activity

The [Jupyter Community Forum](https://discourse.jupyter.org) is a place for Jovyans across the
community to talk about Jupyter tools in interactive computing and how they fit into their
workflows. It's also a place for developers to share ideas, tools, tips, and help one another.

Below are a few updates from activity in the Discourse. For more detailed information about
the activity on the Community Forum, check out these links:

* [The users page](https://discourse.jupyter.org/u) has information about user activity
* [The top posts page](https://discourse.jupyter.org/top) contains a list of top posts, sorted
  by various metrics.

In [100]:
# Get categories for IDs
url = "https://discourse.jupyter.org/site.json"
resp = requests.get(url, headers=api)
category_mapping = {cat['id']: cat['name'] for cat in resp.json()['categories']}
parent_categories = {cat['id']: cat.get("parent_category_id", None) for cat in resp.json()['categories']}

In [101]:
# Base URL to use
url = "https://discourse.jupyter.org/latest.json"

## Topics with lots of likes

"Likes" are a way for community members to say thanks for a helpful post, show their
support for an idea, or generally to share a little positivity with somebody else.
These are topics that have generated lots of likes in recent history.

In [102]:
params = {"order": "likes", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
liked = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(liked))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (98 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (47 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (28 posts)
* [JupyterHub, Binder workshop](https://discourse.jupyter.org/t/jupyterhub-binder-workshop) (14 posts)
* [Help us choose an updated &ldquo;Launch Binder&rdquo; badge!](https://discourse.jupyter.org/t/help-us-choose-an-updated-launch-binder-badge) (21 posts)
* [PyCon 2019 and mybinder.org](https://discourse.jupyter.org/t/pycon-2019-and-mybinder-org) (17 posts)
* [The Binder Federation](https://discourse.jupyter.org/t/the-binder-federation) (14 posts)
* [Jupyter community workshop: proposal on diversity, inclusion and accessibility](https://discourse.jupyter.org/t/jupyter-community-workshop-proposal-on-diversity-inclusion-and-accessibility) (20 posts)
* [Sunset the GitHub repo jupyter/help?](https://discourse.jupyter.org/t/sunset-the-github-repo-jupyter-help) (17 posts)
* [Reviving the all-Jupyter community meetings](https://discourse.jupyter.org/t/reviving-the-all-jupyter-community-meetings) (17 posts)

## Active topics on Discourse

These are topics with lots of activity in recent history.

In [103]:
params = {"order": "posts", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most posts in recent history
posts = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(posts))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (98 posts)
* [Tip: embed custom github content in a Binder link with nbgitpuller](https://discourse.jupyter.org/t/tip-embed-custom-github-content-in-a-binder-link-with-nbgitpuller) (47 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (47 posts)
* [Would a &ldquo;The Littlest Binder&rdquo; be useful?](https://discourse.jupyter.org/t/would-a-the-littlest-binder-be-useful) (31 posts)
* [Plugins of considerations](https://discourse.jupyter.org/t/plugins-of-considerations) (28 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (28 posts)
* [A proposal for JupyterHub communications](https://discourse.jupyter.org/t/a-proposal-for-jupyterhub-communications) (26 posts)
* [Binder/BinderHub Workshops &ndash; Help Wanted!](https://discourse.jupyter.org/t/binder-binderhub-workshops-help-wanted) (25 posts)
* [Rename this category to Jupyter Server?](https://discourse.jupyter.org/t/rename-this-category-to-jupyter-server) (21 posts)
* [Potential collaboration on user research?](https://discourse.jupyter.org/t/potential-collaboration-on-user-research) (21 posts)

In [104]:
counts = counts_from_activity(posts)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:            category_id  bookmarked     parent_category  \
  10       accessibility           2      Special Topics   
  9       Admin requests           1       Site Feedback   
  0                 Meta           5                Meta   
  1        announcements           1                Meta   
  2           JupyterHub           2          JupyterHub   
  6              discuss           2          JupyterHub   
  8   Enterprise Gateway           1  Enterprise Gateway   
  3       Community Chat           3      Community Chat   
  4               Binder           6              Binder   
  7              discuss           6              Binder   
  5            BinderHub           1              Binder   
  
                            parent/category  
  10           Special Topics/accessibility  
  9            Site Feedback/Admin requests  
  0                               Meta/Meta  
  1                      Meta/announcements  
  2                   JupyterHub/JupyterHub  
  6                      JupyterHub/discuss  
  8   Enterprise Gateway/Enterprise Gateway  
  3           Community Chat/Community Chat  
  4                           Binder/Binder  
  7                          Binder/discuss  
  5                        Binder/BinderHub  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['accessibility', 'Admin requests', 'Meta', 'announcements', 'JupyterHub', 'discuss', 'Enterprise Gateway', 'Community Chat', 'Binder', 'discuss', 'BinderHub']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## Recently-created topics

These are topics that were recently created, sorted by the amount of activity
in each one.

In [105]:
params = {"order": "created", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Sort created by the most posted for recently-created posts
created = pd.DataFrame(resp.json()['topic_list']['topics'])
created = created.sort_values('posts_count', ascending=False)
Markdown(topics_to_markdown(created))

* [Creating environment.yml for Python 2 via Binder](https://discourse.jupyter.org/t/creating-environment-yml-for-python-2-via-binder) (11 posts)
* [Why does nb2kg connect to the kernel via EG rather than directly?](https://discourse.jupyter.org/t/why-does-nb2kg-connect-to-the-kernel-via-eg-rather-than-directly) (9 posts)
* [Boilerplate needed for making a Jupyterlab extension using Vue.js](https://discourse.jupyter.org/t/boilerplate-needed-for-making-a-jupyterlab-extension-using-vue-js) (6 posts)
* [Cells not importing modules anymore](https://discourse.jupyter.org/t/cells-not-importing-modules-anymore) (5 posts)
* [Add entries to the filebrowser contextmenu](https://discourse.jupyter.org/t/add-entries-to-the-filebrowser-contextmenu) (4 posts)
* [Can JupyterLab access/use user installed fonts?](https://discourse.jupyter.org/t/can-jupyterlab-access-use-user-installed-fonts) (4 posts)
* [Minimal layout for newbies](https://discourse.jupyter.org/t/minimal-layout-for-newbies) (4 posts)
* [Recent activity across all of Jupyter: a notebook report](https://discourse.jupyter.org/t/recent-activity-across-all-of-jupyter-a-notebook-report) (4 posts)
* [Installing a jupyter notebook extension on Binderhub](https://discourse.jupyter.org/t/installing-a-jupyter-notebook-extension-on-binderhub) (3 posts)
* [Technology choices for new Jupyter.org website](https://discourse.jupyter.org/t/technology-choices-for-new-jupyter-org-website) (3 posts)

In [106]:
counts = counts_from_activity(created)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:            category_id  bookmarked     parent_category  \
  6                  Q&A           1                 Q&A   
  10            Notebook           2            Notebook   
  9            nbconvert           1            Notebook   
  1                 Meta           2                Meta   
  5           JupyterLab           3          JupyterLab   
  2           JupyterHub           0          JupyterHub   
  8           Governance           1          Governance   
  7   Enterprise Gateway           1  Enterprise Gateway   
  0            repo help           1              Binder   
  3               Binder           1              Binder   
  4              discuss           0              Binder   
  
                            parent/category  
  6                                 Q&A/Q&A  
  10                      Notebook/Notebook  
  9                      Notebook/nbconvert  
  1                               Meta/Meta  
  5                   JupyterLab/JupyterLab  
  2                   JupyterHub/JupyterHub  
  8                   Governance/Governance  
  7   Enterprise Gateway/Enterprise Gateway  
  0                        Binder/repo help  
  3                           Binder/Binder  
  4                          Binder/discuss  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['Q&A', 'Notebook', 'nbconvert', 'Meta', 'JupyterLab', 'JupyterHub', 'Governance', 'Enterprise Gateway', 'repo help', 'Binder', 'discuss']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## User activity in Discourse

These are the top posters and top "like-ers" in the Discourse

In [107]:
def plot_user_data(users, column, sort=False):
    plt_data = users.sort_values(column, ascending=False).head(50)
    x = alt.X("username", sort=plt_data['username'].tolist()) if sort is True else 'username'
    ch = alt.Chart(data=plt_data).mark_bar().encode(
        x=x,
        y=column
    )
    return ch

In [108]:
url = "https://discourse.jupyter.org/directory_items.json"
params = {"period": "quarterly", "order": "post_count"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
users = pd.DataFrame(resp.json()['directory_items'])
users['username'] = users['user'].map(lambda a: a['username'])

In [109]:
plot_user_data(users.head(50), 'post_count')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             78    6          213             129         161        1354   
  1             90    3          198             115         134         985   
  2             65   24           16              48          62         622   
  3             83    8           10              59          53         859   
  4             64   12            9              48          47         844   
  5             71  116           21              51          44         610   
  6             78   73           16              31          33        1523   
  7             15  220            1              24          27         189   
  8             62  239            6               7          25         451   
  9             81   30           78              17          24         326   
  10            58   99           43              19          22         533   
  11            19  234            5               3          16          84   
  12            28  485           41              17          16         240   
  15            30   60           12               9          14         338   
  14            31  372            6              13          14         209   
  13            28  419            0               3          14          77   
  16            11  254           13               1          12         175   
  17            62   11           56               9          11         766   
  18            86   77            6               8          11         838   
  19            35  432           28               6          10         272   
  20            10  510            0               6           8          10   
  21            10  390            2               1           8          60   
  22            32  217            9               5           8         177   
  29             5  679            0               8           7          66   
  33            21  333           11               4           7          28   
  32            13  351            0               8           7          56   
  31            18   34            2              14           7         146   
  30            23  281            4               8           7          48   
  25             9  566            0               1           7          29   
  28            10   71            0               4           7          76   
  26            17  227           10               2           7         259   
  24            17  462            0               1           7          73   
  23            14  487            0               0           7          83   
  27            14    9            0               9           7          93   
  37            45  222            2              13           6         128   
  40            25  134            4              15           6         227   
  38            13  145            0               8           6          50   
  39            18  237            1              25           6          51   
  36            44  303            5               3           6         162   
  35            17    4            0              17           6          42   
  34             9  195            0               1           6          33   
  41             6  562            0               0           5          22   
  42            11  247            6              14           5         183   
  43            11  430            2               3           5          41   
  44             6  556            3               6           5          65   
  45             5  565            2               3           5          33   
  46            31  184           14               2           5         260   
  47            41   67            6               7           5         293   
  48            47  298            0               4           5  

**Discourse users sorted by likes given**

In [110]:
plot_user_data(users.head(50), 'likes_given')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             78    6          213             129         161        1354   
  1             90    3          198             115         134         985   
  9             81   30           78              17          24         326   
  17            62   11           56               9          11         766   
  10            58   99           43              19          22         533   
  12            28  485           41              17          16         240   
  19            35  432           28               6          10         272   
  5             71  116           21              51          44         610   
  6             78   73           16              31          33        1523   
  2             65   24           16              48          62         622   
  46            31  184           14               2           5         260   
  16            11  254           13               1          12         175   
  15            30   60           12               9          14         338   
  33            21  333           11               4           7          28   
  26            17  227           10               2           7         259   
  3             83    8           10              59          53         859   
  4             64   12            9              48          47         844   
  22            32  217            9               5           8         177   
  47            41   67            6               7           5         293   
  42            11  247            6              14           5         183   
  49            72  229            6               0           5         141   
  18            86   77            6               8          11         838   
  14            31  372            6              13          14         209   
  8             62  239            6               7          25         451   
  36            44  303            5               3           6         162   
  11            19  234            5               3          16          84   
  40            25  134            4              15           6         227   
  30            23  281            4               8           7          48   
  44             6  556            3               6           5          65   
  31            18   34            2              14           7         146   
  43            11  430            2               3           5          41   
  21            10  390            2               1           8          60   
  45             5  565            2               3           5          33   
  37            45  222            2              13           6         128   
  39            18  237            1              25           6          51   
  7             15  220            1              24          27         189   
  41             6  562            0               0           5          22   
  48            47  298            0               4           5         296   
  35            17    4            0              17           6          42   
  38            13  145            0               8           6          50   
  20            10  510            0               6           8          10   
  34             9  195            0               1           6          33   
  13            28  419            0               3          14          77   
  32            13  351            0               8           7          56   
  29             5  679            0               8           7          66   
  28            10   71            0               4           7          76   
  27            14    9            0               9           7          93   
  24            17  462            0               1           7          73   
  23            14  487            0               0           7  

**Discourse users sorted by likes received**

In [111]:
plot_user_data(users.head(50), 'likes_received')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             78    6          213             129         161        1354   
  1             90    3          198             115         134         985   
  3             83    8           10              59          53         859   
  5             71  116           21              51          44         610   
  2             65   24           16              48          62         622   
  4             64   12            9              48          47         844   
  6             78   73           16              31          33        1523   
  39            18  237            1              25           6          51   
  7             15  220            1              24          27         189   
  10            58   99           43              19          22         533   
  35            17    4            0              17           6          42   
  9             81   30           78              17          24         326   
  12            28  485           41              17          16         240   
  40            25  134            4              15           6         227   
  31            18   34            2              14           7         146   
  42            11  247            6              14           5         183   
  14            31  372            6              13          14         209   
  37            45  222            2              13           6         128   
  17            62   11           56               9          11         766   
  15            30   60           12               9          14         338   
  27            14    9            0               9           7          93   
  18            86   77            6               8          11         838   
  32            13  351            0               8           7          56   
  30            23  281            4               8           7          48   
  29             5  679            0               8           7          66   
  38            13  145            0               8           6          50   
  47            41   67            6               7           5         293   
  8             62  239            6               7          25         451   
  20            10  510            0               6           8          10   
  44             6  556            3               6           5          65   
  19            35  432           28               6          10         272   
  22            32  217            9               5           8         177   
  28            10   71            0               4           7          76   
  33            21  333           11               4           7          28   
  48            47  298            0               4           5         296   
  45             5  565            2               3           5          33   
  36            44  303            5               3           6         162   
  13            28  419            0               3          14          77   
  11            19  234            5               3          16          84   
  43            11  430            2               3           5          41   
  26            17  227           10               2           7         259   
  46            31  184           14               2           5         260   
  25             9  566            0               1           7          29   
  34             9  195            0               1           6          33   
  24            17  462            0               1           7          73   
  21            10  390            2               1           8          60   
  16            11  254           13               1          12         175   
  41             6  562            0               0           5          22   
  23            14  487            0               0           7  

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [112]:
# Define colors we'll use for GitHub membership
import seaborn as sns
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_colors = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_colors]

In [113]:
from datetime import timedelta

orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/ipython](https://github.com/ipython)
* [github.com/binder-examples](https://github.com/binder-examples)
* [github.com/nteract](https://github.com/nteract)

In [114]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-05-16** to **2019-07-15**

In [115]:
responses = []
for org in github_orgs:
    query_prs = f"is:merged user:{org} archived:false closed:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
merged = pd.concat([ii.data for ii in responses])

Found 223 items, which will take 5 pages
Found 247 items, which will take 5 pages
Found 228 items, which will take 5 pages
Found 93 items, which will take 2 pages
Found 53 items, which will take 2 pages
Found 3 items, which will take 1 pages
Found 137 items, which will take 3 pages


In [116]:
# Add a PR-specific field for closed PRs
merged['closed_by'] = merged['mergedBy'].map(lambda a: a['login'])
merged['closed_by_url'] = merged['mergedBy'].map(lambda a: a['url'])

In [117]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)

## Commentors

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [118]:
alt.Chart(data=prs_by_repo, title="Merged PRs in the last 30 days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                 org                           repo  author
  73          nteract                       hydrogen      39
  71          nteract                      bookstore      36
  76          nteract                        nteract      26
  78          nteract                      papermill      16
  77          nteract                     nteract.io       9
  79          nteract                      scrapbook       3
  72          nteract                       commuter       2
  75          nteract                 minimal-plotly       2
  81          nteract                  vega-embed-v3       2
  74          nteract                   kernel-relay       1
  80          nteract                           vdom       1
  56       jupyterlab                     jupyterlab     166
  60       jupyterlab                 jupyterlab-git      13
  67       jupyterlab              jupyterlab_server       9
  59       jupyterlab                jupyterlab-demo       8
  55       jupyterlab              jupyter-renderers       7
  57       jupyterlab          jupyterlab-commenting       4
  58       jupyterlab       jupyterlab-data-explorer       4
  70       jupyterlab             theme-cookiecutter       3
  54       jupyterlab      extension-cookiecutter-ts       2
  61       jupyterlab              jupyterlab-github       2
  65       jupyterlab                 jupyterlab-toc       2
  69       jupyterlab  scipy2019-jupyterlab-tutorial       2
  53       jupyterlab      extension-cookiecutter-js       1
  62       jupyterlab               jupyterlab-latex       1
  63       jupyterlab    jupyterlab-metadata-service       1
  64       jupyterlab           jupyterlab-telemetry       1
  66       jupyterlab            jupyterlab_pygments       1
  68       jupyterlab     mimerender-cookiecutter-ts       1
  42       jupyterhub            mybinder.org-deploy      90
  ..              ...                            ...     ...
  29  jupyter-widgets                      pythreejs       5
  27  jupyter-widgets             jupyterlab-sidecar       1
  28  jupyter-widgets                   midicontrols       1
  30  jupyter-widgets                     traittypes       1
  32  jupyter-widgets            widget-cookiecutter       1
  21          jupyter                       notebook      50
  19          jupyter                       nbgrader      43
  23          jupyter                    repo2docker      25
  8           jupyter             enterprise_gateway      24
  17          jupyter                      nbconvert      18
  24          jupyter                      telemetry      15
  13          jupyter                 jupyter_client      12
  10          jupyter                   jupyter-book      11
  22          jupyter                      qtconsole       9
  11          jupyter                 jupyter-sphinx       8
  15          jupyter                 jupyter_server       8
  14          jupyter                   jupyter_core       7
  12          jupyter              jupyter.github.io       5
  7           jupyter                  docker-stacks       4
  20          jupyter                       nbviewer       4
  18          jupyter                         nbdime       2
  9           jupyter                        jupyter       1
  16          jupyter                          nb2kg       1
  5           ipython                        ipython      42
  6           ipython                      traitlets       5
  3           ipython                      ipykernel       4
  4           ipython                    ipyparallel       2
  0   binder-examples                     data-quilt       1
  1   binder-examples             getting-data-xroot       1
  2   binder-examples                          latex       1
  
  [82 rows x 3 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['hydrogen', 'bookstore', 'nterac

### A list of merged PRs by project

In [119]:
from ipywidgets import widgets, HTML
from ipywidgets import HTML
from markdown import markdown 

In [120]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['username']
            user_url = pr['username_url']
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['closed_by']
            pr_closedby_url = pr['closed_by_url']
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

Authoring and merging stats by repo

In [121]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'username']).count()['author'].reset_index().rename(columns={'author': "authored"})
closed_by_repo = merged.groupby(['org', 'repo', 'closed_by']).count()['author'].reset_index().rename(columns={'author': "closed", "closed_by": "username"})

merged_stats = pd.merge(closed_by_repo, merged_by_repo, on=['org', 'repo', 'username'], how='outer')
merged_stats = pd.melt(merged_stats, id_vars=['org', 'repo', 'username'], var_name="kind", value_name="count")
merged_stats = merged_stats.replace(np.nan, 0)

In [122]:
for ii, (iorg, idata) in enumerate(merged_stats.groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='count',
        color='kind',
    )
    display(ch)

Chart({
  data:                  org                repo         username      kind  count
  0    binder-examples          data-quilt          betatim    closed    1.0
  1    binder-examples  getting-data-xroot        choldgraf    closed    1.0
  2    binder-examples               latex        choldgraf    closed    1.0
  177  binder-examples          data-quilt      matheusmota    closed    0.0
  178  binder-examples  getting-data-xroot  matthewfeickert    closed    0.0
  179  binder-examples               latex        yuvipanda    closed    0.0
  456  binder-examples          data-quilt          betatim  authored    0.0
  457  binder-examples  getting-data-xroot        choldgraf  authored    0.0
  458  binder-examples               latex        choldgraf  authored    0.0
  633  binder-examples          data-quilt      matheusmota  authored    1.0
  634  binder-examples  getting-data-xroot  matthewfeickert  authored    1.0
  635  binder-examples               latex        yuvipanda  authored    1.0,
  encoding: FacetedEncoding({
    color: Color({
      field: 'kind',
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'count',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Merges for binder-examples in the last 60 days',
  width: 1000
})

Chart({
  data:          org         repo        username      kind  count
  3    ipython    ipykernel       blink1073    closed    1.0
  4    ipython    ipykernel           minrk    closed    3.0
  5    ipython  ipyparallel           minrk    closed    2.0
  6    ipython      ipython         Carreau    closed   42.0
  7    ipython    traitlets        rmorshea    closed    5.0
  180  ipython    ipykernel         Chilipp    closed    0.0
  181  ipython    ipykernel         hroncok    closed    0.0
  182  ipython    ipykernel        steff456    closed    0.0
  183  ipython  ipyparallel           Jamim    closed    0.0
  184  ipython  ipyparallel          ellert    closed    0.0
  185  ipython      ipython  LucianaMarques    closed    0.0
  186  ipython      ipython      OriolAbril    closed    0.0
  187  ipython      ipython       Pragnya02    closed    0.0
  188  ipython      ipython            alok    closed    0.0
  189  ipython      ipython         anntzer    closed    0.0
  190  ipython      ipython        bashtage    closed    0.0
  191  ipython      ipython         blueyed    closed    0.0
  192  ipython      ipython  brandonwillard    closed    0.0
  193  ipython      ipython          daharn    closed    0.0
  194  ipython      ipython        eobrie16    closed    0.0
  195  ipython      ipython      gooverdian    closed    0.0
  196  ipython      ipython         hroncok    closed    0.0
  197  ipython      ipython           k-zen    closed    0.0
  198  ipython      ipython          kmaork    closed    0.0
  199  ipython      ipython      pierstitus    closed    0.0
  200  ipython      ipython           rgson    closed    0.0
  201  ipython      ipython         sunpoet    closed    0.0
  202  ipython      ipython       tacaswell    closed    0.0
  203  ipython    traitlets           Jamim    closed    0.0
  204  ipython    traitlets    danielballan    closed    0.0
  ..       ...          ...             ...       ...    ...
  461  ipython  ipyparallel           minrk  authored    0.0
  462  ipython      ipython         Carreau  authored   21.0
  463  ipython    traitlets        rmorshea  authored    0.0
  636  ipython    ipykernel         Chilipp  authored    1.0
  637  ipython    ipykernel         hroncok  authored    1.0
  638  ipython    ipykernel        steff456  authored    1.0
  639  ipython  ipyparallel           Jamim  authored    1.0
  640  ipython  ipyparallel          ellert  authored    1.0
  641  ipython      ipython  LucianaMarques  authored    2.0
  642  ipython      ipython      OriolAbril  authored    2.0
  643  ipython      ipython       Pragnya02  authored    1.0
  644  ipython      ipython            alok  authored    2.0
  645  ipython      ipython         anntzer  authored    1.0
  646  ipython      ipython        bashtage  authored    1.0
  647  ipython      ipython         blueyed  authored    1.0
  648  ipython      ipython  brandonwillard  authored    1.0
  649  ipython      ipython          daharn  authored    1.0
  650  ipython      ipython        eobrie16  authored    1.0
  651  ipython      ipython      gooverdian  authored    1.0
  652  ipython      ipython         hroncok  authored    1.0
  653  ipython      ipython           k-zen  authored    1.0
  654  ipython      ipython          kmaork  authored    1.0
  655  ipython      ipython      pierstitus  authored    1.0
  656  ipython      ipython           rgson  authored    1.0
  657  ipython      ipython         sunpoet  authored    1.0
  658  ipython      ipython       tacaswell  authored    1.0
  659  ipython    traitlets           Jamim  authored    2.0
  660  ipython    traitlets    danielballan  authored    1.0
  661  ipython    traitlets   frenzymadness  authored    1.0
  662  ipython    traitlets     kevin-bates  authored    1.0
  
  [64 rows x 5 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'kind',
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({

Chart({
  data:          org                repo       username      kind  count
  8    jupyter       docker-stacks          minrk    closed    2.0
  9    jupyter       docker-stacks        parente    closed    2.0
  10   jupyter  enterprise_gateway      akchinSTC    closed    2.0
  11   jupyter  enterprise_gateway    kevin-bates    closed   16.0
  12   jupyter  enterprise_gateway       lresende    closed    6.0
  13   jupyter             jupyter        parente    closed    1.0
  14   jupyter        jupyter-book      choldgraf    closed   11.0
  15   jupyter      jupyter-sphinx  SylvainCorlay    closed    1.0
  16   jupyter      jupyter-sphinx       akhmerov    closed    6.0
  17   jupyter      jupyter-sphinx       jbweston    closed    1.0
  18   jupyter   jupyter.github.io           Ruv7    closed    1.0
  19   jupyter   jupyter.github.io         afshin    closed    1.0
  20   jupyter   jupyter.github.io      ellisonbg    closed    1.0
  21   jupyter   jupyter.github.io     jasongrout    closed    2.0
  22   jupyter      jupyter_client          MSeal    closed    1.0
  23   jupyter      jupyter_client  SylvainCorlay    closed    4.0
  24   jupyter      jupyter_client      blink1073    closed    1.0
  25   jupyter      jupyter_client          minrk    closed    1.0
  26   jupyter      jupyter_client         mpacer    closed    1.0
  27   jupyter      jupyter_client         rgbkrk    closed    4.0
  28   jupyter        jupyter_core          minrk    closed    1.0
  29   jupyter        jupyter_core         rgbkrk    closed    1.0
  30   jupyter        jupyter_core      takluyver    closed    5.0
  31   jupyter      jupyter_server  SylvainCorlay    closed    3.0
  32   jupyter      jupyter_server      blink1073    closed    1.0
  33   jupyter      jupyter_server         ivanov    closed    1.0
  34   jupyter      jupyter_server       rolweber    closed    3.0
  35   jupyter               nb2kg    kevin-bates    closed    1.0
  36   jupyter           nbconvert          MSeal    closed   13.0
  37   jupyter           nbconvert       t-makaro    closed    4.0
  ..       ...                 ...            ...       ...    ...
  716  jupyter            notebook        betatim  authored    1.0
  717  jupyter            notebook    dalanicolai  authored    1.0
  718  jupyter            notebook        ednut15  authored    3.0
  719  jupyter            notebook    fabihaahmed  authored    3.0
  720  jupyter            notebook        gnestor  authored    1.0
  721  jupyter            notebook         hdtee1  authored    1.0
  722  jupyter            notebook  joshuazeltser  authored   16.0
  723  jupyter            notebook    kevin-bates  authored    3.0
  724  jupyter            notebook         mgeier  authored    1.0
  725  jupyter            notebook        minho42  authored    1.0
  726  jupyter            notebook       mtd91429  authored    1.0
  727  jupyter            notebook       saraedum  authored    1.0
  728  jupyter            notebook     ssanderson  authored    1.0
  729  jupyter            notebook       t-makaro  authored    1.0
  730  jupyter            notebook       taohan16  authored    1.0
  731  jupyter            notebook         tholoz  authored    1.0
  732  jupyter            notebook         tmetzl  authored    1.0
  733  jupyter            notebook     vibvibgyor  authored    1.0
  734  jupyter           qtconsole          ajtam  authored    1.0
  735  jupyter           qtconsole       dalthviz  authored    2.0
  736  jupyter           qtconsole       impact27  authored    1.0
  737  jupyter         repo2docker     Xarthisius  authored    1.0
  738  jupyter         repo2docker   consideRatio  authored    1.0
  739  jupyter         repo2docker   davidanthoff  authored    1.0
  740  jupyter         repo2docker     davidrpugh  authored    1.0
  741  jupyter         repo2docker      eexwhyzee  authored    1.0
  742  jupyter         repo2docker      fmaussion  authored    1.0
  743  jupyter         repo2docke

Chart({
  data:                  org                 repo          username      kind  count
  56   jupyter-widgets           ipyleaflet     SylvainCorlay    closed    3.0
  57   jupyter-widgets           ipyleaflet       martinRenou    closed   21.0
  58   jupyter-widgets           ipywidgets     SylvainCorlay    closed   20.0
  59   jupyter-widgets           ipywidgets        jasongrout    closed    7.0
  60   jupyter-widgets           ipywidgets           vidartf    closed    3.0
  61   jupyter-widgets   jupyterlab-sidecar     SylvainCorlay    closed    1.0
  62   jupyter-widgets         midicontrols        jasongrout    closed    1.0
  63   jupyter-widgets            pythreejs           vidartf    closed    5.0
  64   jupyter-widgets           traittypes     SylvainCorlay    closed    1.0
  65   jupyter-widgets             tutorial     SylvainCorlay    closed    1.0
  66   jupyter-widgets             tutorial   maartenbreddels    closed    1.0
  67   jupyter-widgets             tutorial       martinRenou    closed    4.0
  68   jupyter-widgets             tutorial           mwcraig    closed   24.0
  69   jupyter-widgets  widget-cookiecutter           vidartf    closed    1.0
  290  jupyter-widgets           ipyleaflet         Juanlu001    closed    0.0
  291  jupyter-widgets           ipyleaflet      VasavanThiru    closed    0.0
  292  jupyter-widgets           ipyleaflet          deeplook    closed    0.0
  293  jupyter-widgets           ipyleaflet             jmp75    closed    0.0
  294  jupyter-widgets           ipyleaflet             jtpio    closed    0.0
  295  jupyter-widgets           ipyleaflet         kgiacobbi    closed    0.0
  296  jupyter-widgets           ipyleaflet          mpdwulit    closed    0.0
  297  jupyter-widgets           ipywidgets             Jamim    closed    0.0
  298  jupyter-widgets           ipywidgets         Juanlu001    closed    0.0
  299  jupyter-widgets           ipywidgets          akhmerov    closed    0.0
  300  jupyter-widgets           ipywidgets             blois    closed    0.0
  301  jupyter-widgets           ipywidgets           edolley    closed    0.0
  302  jupyter-widgets           ipywidgets            ivanov    closed    0.0
  303  jupyter-widgets           ipywidgets             jtpio    closed    0.0
  304  jupyter-widgets           ipywidgets            lheagy    closed    0.0
  305  jupyter-widgets           ipywidgets       martinRenou    closed    0.0
  ..               ...                  ...               ...       ...    ...
  523  jupyter-widgets             tutorial       martinRenou  authored   17.0
  524  jupyter-widgets             tutorial           mwcraig  authored   10.0
  525  jupyter-widgets  widget-cookiecutter           vidartf  authored    0.0
  746  jupyter-widgets           ipyleaflet         Juanlu001  authored    1.0
  747  jupyter-widgets           ipyleaflet      VasavanThiru  authored    4.0
  748  jupyter-widgets           ipyleaflet          deeplook  authored    1.0
  749  jupyter-widgets           ipyleaflet             jmp75  authored    1.0
  750  jupyter-widgets           ipyleaflet             jtpio  authored    1.0
  751  jupyter-widgets           ipyleaflet         kgiacobbi  authored    5.0
  752  jupyter-widgets           ipyleaflet          mpdwulit  authored    1.0
  753  jupyter-widgets           ipywidgets             Jamim  authored    1.0
  754  jupyter-widgets           ipywidgets         Juanlu001  authored    1.0
  755  jupyter-widgets           ipywidgets          akhmerov  authored    1.0
  756  jupyter-widgets           ipywidgets             blois  authored    1.0
  757  jupyter-widgets           ipywidgets           edolley  authored    1.0
  758  jupyter-widgets           ipywidgets            ivanov  authored    1.0
  759  jupyter-widgets           ipywidgets             jtpio  authored    1.0
  760  jupyter-widgets           ipywidgets            lheagy  authored    1.0
  761  jupyter-widgets           ipy

Chart({
  data:             org                     repo        username      kind  count
  70   jupyterhub     alabaster-jupyterhub         betatim    closed    1.0
  71   jupyterhub     alabaster-jupyterhub       choldgraf    closed    8.0
  72   jupyterhub             batchspawner      mbmilligan    closed    7.0
  73   jupyterhub                   binder         betatim    closed    1.0
  74   jupyterhub                binderhub         betatim    closed   11.0
  75   jupyterhub                binderhub       choldgraf    closed    2.0
  76   jupyterhub                binderhub    consideRatio    closed    3.0
  77   jupyterhub                binderhub           minrk    closed    1.0
  78   jupyterhub                binderhub       sgibson91    closed    1.0
  79   jupyterhub                binderhub       yuvipanda    closed    1.0
  80   jupyterhub               chartpress           minrk    closed    1.0
  81   jupyterhub               chartpress       yuvipanda    closed    1.0
  82   jupyterhub            dockerspawner           minrk    closed    1.0
  83   jupyterhub     jupyter-server-proxy         betatim    closed    1.0
  84   jupyterhub     jupyter-server-proxy       yuvipanda    closed    2.0
  85   jupyterhub               jupyterhub         betatim    closed    3.0
  86   jupyterhub               jupyterhub           minrk    closed    6.0
  87   jupyterhub               jupyterhub       yuvipanda    closed    3.0
  88   jupyterhub         ltiauthenticator       yuvipanda    closed    1.0
  89   jupyterhub      mybinder.org-deploy         betatim    closed   78.0
  90   jupyterhub      mybinder.org-deploy       choldgraf    closed    4.0
  91   jupyterhub      mybinder.org-deploy    consideRatio    closed    1.0
  92   jupyterhub      mybinder.org-deploy          henchc    closed    5.0
  93   jupyterhub      mybinder.org-deploy           minrk    closed    1.0
  94   jupyterhub      mybinder.org-deploy       sgibson91    closed    1.0
  95   jupyterhub              nbgitpuller       choldgraf    closed    2.0
  96   jupyterhub              nbgitpuller       yuvipanda    closed    3.0
  97   jupyterhub           oauthenticator         betatim    closed    1.0
  98   jupyterhub           oauthenticator           minrk    closed    3.0
  99   jupyterhub              sudospawner    consideRatio    closed    1.0
  ..          ...                      ...             ...       ...    ...
  801  jupyterhub              nbgitpuller     parmentelat  authored    1.0
  802  jupyterhub           oauthenticator    DentonGentry  authored    1.0
  803  jupyterhub           oauthenticator          nifuki  authored    1.0
  804  jupyterhub           oauthenticator       sunnielyu  authored    1.0
  805  jupyterhub              sudospawner         lumbric  authored    1.0
  806  jupyterhub           systemdspawner       mortbauer  authored    1.0
  807  jupyterhub             team-compass         Zsailer  authored    2.0
  808  jupyterhub             team-compass          lheagy  authored    1.0
  809  jupyterhub             team-compass      ryanlovett  authored    1.0
  810  jupyterhub  the-littlest-jupyterhub         JuanCab  authored    1.0
  811  jupyterhub  the-littlest-jupyterhub     cornhundred  authored    1.0
  812  jupyterhub  the-littlest-jupyterhub           jtpio  authored    7.0
  813  jupyterhub  the-littlest-jupyterhub         lumbric  authored    1.0
  814  jupyterhub  the-littlest-jupyterhub            owah  authored    1.0
  815  jupyterhub  the-littlest-jupyterhub        pbugnion  authored    1.0
  816  jupyterhub  the-littlest-jupyterhub         staeiou  authored    2.0
  817  jupyterhub  the-littlest-jupyterhub        trallard  authored    1.0
  818  jupyterhub         tmpauthenticator         mohirio  authored    1.0
  819  jupyterhub            traefik-proxy  GeorgianaElena  authored    5.0
  820  jupyterhub              wrapspawner        cmd-ntrf  authored    2.0
  821  jupyterhub   zero-to-jupyte

Chart({
  data:             org                           repo         username      kind  \
  115  jupyterlab      extension-cookiecutter-js            jtpio    closed   
  116  jupyterlab      extension-cookiecutter-ts        blink1073    closed   
  117  jupyterlab              jupyter-renderers          gnestor    closed   
  118  jupyterlab              jupyter-renderers       ian-r-rose    closed   
  119  jupyterlab                     jupyterlab           afshin    closed   
  120  jupyterlab                     jupyterlab         aschlaep    closed   
  121  jupyterlab                     jupyterlab        blink1073    closed   
  122  jupyterlab                     jupyterlab        ellisonbg    closed   
  123  jupyterlab                     jupyterlab       fcollonval    closed   
  124  jupyterlab                     jupyterlab          gnestor    closed   
  125  jupyterlab                     jupyterlab       ian-r-rose    closed   
  126  jupyterlab                     jupyterlab           ivanov    closed   
  127  jupyterlab                     jupyterlab       jasongrout    closed   
  128  jupyterlab                     jupyterlab            jtpio    closed   
  129  jupyterlab                     jupyterlab   saulshanabrook    closed   
  130  jupyterlab                     jupyterlab       telamonian    closed   
  131  jupyterlab                     jupyterlab          vidartf    closed   
  132  jupyterlab          jupyterlab-commenting           acu192    closed   
  133  jupyterlab          jupyterlab-commenting           hoo761    closed   
  134  jupyterlab       jupyterlab-data-explorer   saulshanabrook    closed   
  135  jupyterlab                jupyterlab-demo      dharmaquark    closed   
  136  jupyterlab                jupyterlab-demo       ian-r-rose    closed   
  137  jupyterlab                jupyterlab-demo       jasongrout    closed   
  138  jupyterlab                 jupyterlab-git            dleen    closed   
  139  jupyterlab                 jupyterlab-git       jaipreet-s    closed   
  140  jupyterlab                 jupyterlab-git   saulshanabrook    closed   
  141  jupyterlab              jupyterlab-github       ian-r-rose    closed   
  142  jupyterlab               jupyterlab-latex       ian-r-rose    closed   
  143  jupyterlab    jupyterlab-metadata-service           acu192    closed   
  144  jupyterlab           jupyterlab-telemetry          Zsailer    closed   
  ..          ...                            ...              ...       ...   
  860  jupyterlab                     jupyterlab        nscozzaro  authored   
  861  jupyterlab                     jupyterlab      parmentelat  authored   
  862  jupyterlab                     jupyterlab          reevejd  authored   
  863  jupyterlab                     jupyterlab         tobiasjj  authored   
  864  jupyterlab                     jupyterlab          tslaton  authored   
  865  jupyterlab                     jupyterlab        yuvipanda  authored   
  866  jupyterlab                     jupyterlab          zerline  authored   
  867  jupyterlab          jupyterlab-commenting       dependabot  authored   
  868  jupyterlab       jupyterlab-data-explorer       telamonian  authored   
  869  jupyterlab                jupyterlab-demo    SylvainCorlay  authored   
  870  jupyterlab                jupyterlab-demo        TheBear44  authored   
  871  jupyterlab                 jupyterlab-git    danielgruesso  authored   
  872  jupyterlab                 jupyterlab-git       dependabot  authored   
  873  jupyterlab                 jupyterlab-git       fcollonval  authored   
  874  jupyterlab                 jupyterlab-git     neelamgehlot  authored   
  875  jupyterlab                 jupyterlab-git        urihoenig  authored   
  876  jupyterlab              jupyterlab-github       dependabot  authored   
  877  jupyterlab               jupyterlab-latex       dependabot  authored   
  878  jupyterlab           jupyterl

Chart({
  data:          org            repo        username      kind  count
  153  nteract       bookstore           MSeal    closed    3.0
  154  nteract       bookstore          mpacer    closed    8.0
  155  nteract       bookstore          rgbkrk    closed    6.0
  156  nteract       bookstore        willingc    closed   19.0
  157  nteract        commuter    captainsafia    closed    2.0
  158  nteract        hydrogen      BenRussert    closed   21.0
  159  nteract        hydrogen        aviatesk    closed   10.0
  160  nteract        hydrogen        renovate    closed    4.0
  161  nteract        hydrogen  wadethestealth    closed    4.0
  162  nteract    kernel-relay    captainsafia    closed    1.0
  163  nteract  minimal-plotly          rgbkrk    closed    2.0
  164  nteract         nteract      BenRussert    closed    8.0
  165  nteract         nteract    captainsafia    closed    7.0
  166  nteract         nteract          emeeks    closed    1.0
  167  nteract         nteract          rgbkrk    closed   10.0
  168  nteract      nteract.io        renovate    closed    8.0
  169  nteract      nteract.io          rgbkrk    closed    1.0
  170  nteract       papermill           MSeal    closed   11.0
  171  nteract       papermill          rgbkrk    closed    4.0
  172  nteract       papermill        willingc    closed    1.0
  173  nteract       scrapbook           MSeal    closed    2.0
  174  nteract       scrapbook          rgbkrk    closed    1.0
  175  nteract            vdom         gnestor    closed    1.0
  176  nteract   vega-embed-v3  CrystallineCat    closed    2.0
  434  nteract        commuter      dependabot    closed    0.0
  435  nteract        hydrogen     JeremyStepp    closed    0.0
  436  nteract        hydrogen        hoishing    closed    0.0
  437  nteract        hydrogen       jwrussert    closed    0.0
  438  nteract    kernel-relay      BenRussert    closed    0.0
  439  nteract  minimal-plotly        renovate    closed    0.0
  ..       ...             ...             ...       ...    ...
  625  nteract      nteract.io          rgbkrk  authored    0.0
  626  nteract       papermill           MSeal  authored    2.0
  627  nteract       papermill          rgbkrk  authored    1.0
  628  nteract       papermill        willingc  authored    0.0
  629  nteract       scrapbook           MSeal  authored    1.0
  630  nteract       scrapbook          rgbkrk  authored    0.0
  631  nteract            vdom         gnestor  authored    1.0
  632  nteract   vega-embed-v3  CrystallineCat  authored    2.0
  890  nteract        commuter      dependabot  authored    2.0
  891  nteract        hydrogen     JeremyStepp  authored    1.0
  892  nteract        hydrogen        hoishing  authored    1.0
  893  nteract        hydrogen       jwrussert  authored    1.0
  894  nteract    kernel-relay      BenRussert  authored    1.0
  895  nteract  minimal-plotly        renovate  authored    2.0
  896  nteract         nteract         and0111  authored    2.0
  897  nteract         nteract      dylanross3  authored    1.0
  898  nteract         nteract        renovate  authored    5.0
  899  nteract         nteract        rganardi  authored    2.0
  900  nteract         nteract  wadethestealth  authored    1.0
  901  nteract       papermill     RoelBrouwer  authored    1.0
  902  nteract       papermill  VincentBLortie  authored    1.0
  903  nteract       papermill   aabdullah-bos  authored    1.0
  904  nteract       papermill       casperdcl  authored    3.0
  905  nteract       papermill    duarteocarmo  authored    3.0
  906  nteract       papermill        khourhin  authored    1.0
  907  nteract       papermill          mwouts  authored    1.0
  908  nteract       papermill        trallard  authored    1.0
  909  nteract       papermill          vizhur  authored    1.0
  910  nteract       scrapbook      matthiasdv  authored    1.0
  911  nteract       scrapbook        trallard  authored    1.0
  
  [92 rows x 5 

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [123]:
responses = []
for org in github_orgs:
    query_prs = f"is:open is:issue user:{org} archived:false created:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
created = pd.concat([ii.data for ii in responses])
created['n_comments'] = created['comments'].map(lambda a: len(a['edges']))

Found 205 items, which will take 5 pages
Found 195 items, which will take 4 pages
Found 199 items, which will take 4 pages
Found 52 items, which will take 2 pages
Found 46 items, which will take 1 pages
Found 2 items, which will take 1 pages
Found 90 items, which will take 2 pages


In [124]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                           repo  number  \
  0   binder-examples                     demo-julia       1   
  1   binder-examples                   jupyter-rise       1   
  2           ipython                      ipykernel       6   
  3           ipython                    ipyparallel       3   
  4           ipython                        ipython      36   
  5           ipython               ipython-in-depth       1   
  6           jupyter                  accessibility       1   
  7           jupyter                     dashboards       1   
  8           jupyter                         design       1   
  9           jupyter                  docker-stacks      14   
  10          jupyter             enterprise_gateway       5   
  11          jupyter                        jupyter       3   
  12          jupyter                   jupyter-book      19   
  13          jupyter                  jupyter-drive       1   
  14          jupyter                 jupyter-sphinx       9   
  15          jupyter                 jupyter_client       6   
  16          jupyter                   jupyter_core       1   
  17          jupyter            jupyter_kernel_test       1   
  18          jupyter                 jupyter_server       9   
  19          jupyter                 kernel_gateway       1   
  20          jupyter                          nb2kg       1   
  21          jupyter                      nbconvert      12   
  22          jupyter                         nbdime       6   
  23          jupyter                       nbgrader      24   
  24          jupyter                       nbviewer       3   
  25          jupyter                       notebook      55   
  26          jupyter                      qtconsole       2   
  27          jupyter                    repo2docker      19   
  28          jupyter                       testpath       1   
  29  jupyter-widgets                     ipyleaflet       7   
  ..              ...                            ...     ...   
  59       jupyterhub         zero-to-jupyterhub-k8s      25   
  60       jupyterlab      extension-cookiecutter-ts       1   
  61       jupyterlab              jupyter-renderers       2   
  62       jupyterlab                     jupyterlab     152   
  63       jupyterlab            jupyterlab-celltags       3   
  64       jupyterlab          jupyterlab-commenting      12   
  65       jupyterlab       jupyterlab-data-explorer      11   
  66       jupyterlab                 jupyterlab-git       7   
  67       jupyterlab        jupyterlab-google-drive       1   
  68       jupyterlab    jupyterlab-metadata-service       1   
  69       jupyterlab           jupyterlab-telemetry       2   
  70       jupyterlab                 jupyterlab-toc       1   
  71       jupyterlab                jupyterlab_apod       1   
  72       jupyterlab                 jupyterlab_app       1   
  73       jupyterlab  scipy2018-jupyterlab-tutorial       1   
  74       jupyterlab                   team-compass       3   
  75          nteract                      bookstore      14   
  76          nteract                    coffee_boat       1   
  77          nteract                       commuter       1   
  78          nteract             commuter-on-glitch       1   
  79          nteract                             dx       3   
  80          nteract                       hydrogen      15   
  81          nteract                meeting-minutes       1   
  82          nteract                 minimal-plotly       1   
  83          nteract                        nteract      27   
  84          nteract                     nteract.io       1   
  85          nteract                      papermill      12   
  86          nteract                      scrapbook       3   
  87          nteract                       semiotic       9   
  88          nteract                           vdom       1   
  
               

In [125]:
responses = []
for org in github_orgs:
    query_prs = f"is:issue user:{org} closed:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
closed = pd.concat([ii.data for ii in responses])
closed['n_comments'] = closed['comments'].map(lambda a: len(a['edges']))

Found 143 items, which will take 3 pages
Found 218 items, which will take 5 pages
Found 219 items, which will take 5 pages
Found 53 items, which will take 2 pages
Found 33 items, which will take 1 pages
Found 1 items, which will take 1 pages
Found 165 items, which will take 4 pages


In [126]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                       repo  number  \
  0   binder-examples                      bokeh       1   
  1           ipython                  ipykernel       3   
  2           ipython                    ipython      28   
  3           ipython                  traitlets       2   
  4           jupyter              atom-notebook       1   
  5           jupyter              docker-stacks       6   
  6           jupyter         enterprise_gateway      21   
  7           jupyter                    jupyter       4   
  8           jupyter               jupyter-book      12   
  9           jupyter             jupyter-sphinx      10   
  10          jupyter          jupyter.github.io       1   
  11          jupyter             jupyter_client       4   
  12          jupyter               jupyter_core       9   
  13          jupyter             jupyter_server       8   
  14          jupyter                      nb2kg       1   
  15          jupyter                  nbconvert      19   
  16          jupyter                     nbdime       1   
  17          jupyter                   nbgrader      39   
  18          jupyter                   nbviewer       4   
  19          jupyter                   notebook      53   
  20          jupyter                  qtconsole       6   
  21          jupyter                repo2docker      15   
  22          jupyter                  telemetry       3   
  23          jupyter                  terminado       1   
  24  jupyter-widgets                 ipyleaflet      14   
  25  jupyter-widgets                 ipywidgets      31   
  26  jupyter-widgets         jupyterlab-sidecar       1   
  27  jupyter-widgets               midicontrols       1   
  28  jupyter-widgets                  pythreejs       3   
  29  jupyter-widgets                   tutorial       3   
  ..              ...                        ...     ...   
  34       jupyterhub       jupyter-server-proxy       2   
  35       jupyterhub                 jupyterhub      22   
  36       jupyterhub   jupyterhub-deploy-docker       1   
  37       jupyterhub                kubespawner       2   
  38       jupyterhub          ldapauthenticator       1   
  39       jupyterhub        mybinder.org-deploy      11   
  40       jupyterhub                nbgitpuller      10   
  41       jupyterhub             oauthenticator       1   
  42       jupyterhub               team-compass      11   
  43       jupyterhub    the-littlest-jupyterhub      48   
  44       jupyterhub           tmpauthenticator       1   
  45       jupyterhub              traefik-proxy       5   
  46       jupyterhub     zero-to-jupyterhub-k8s      15   
  47       jupyterlab  extension-cookiecutter-js       1   
  48       jupyterlab          jupyter-renderers       7   
  49       jupyterlab                 jupyterlab     186   
  50       jupyterlab   jupyterlab-data-explorer       7   
  51       jupyterlab            jupyterlab-demo       2   
  52       jupyterlab             jupyterlab-git      12   
  53       jupyterlab          jupyterlab-github       1   
  54       jupyterlab             jupyterlab-toc       2   
  55       jupyterlab          jupyterlab_server       1   
  56          nteract                  bookstore      12   
  57          nteract                   commuter       1   
  58          nteract                   hydrogen      98   
  59          nteract                    nteract      35   
  60          nteract                  papermill      10   
  61          nteract                  scrapbook       1   
  62          nteract                   semiotic       5   
  63          nteract                       vdom       3   
  
                                  org/repo  
  0                  binder-examples/bokeh  
  1                      ipython/ipykernel  
  2                        ipython/ipython  
  3                      ipython/traitlets  
  4                  jupyter/atom-note

### A list of recent issues

In [127]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(created.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for (org, repo), issues in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, issue in issues.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['username']
            user_url = issue['username_url']
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)
    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 60 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [128]:
comments = []
for items in [created, merged, closed]:
    for ii, irow in items.iterrows():
        icomments = [ii['node'] for ii in irow['comments']['edges']]
        icomments = pd.DataFrame(icomments)
        icomments['repo'] = irow['repo']
        icomments['org'] = irow['org']
        comments.append(icomments)
comments = pd.concat(comments, sort=False)
comments = comments.query("createdAt > @start_date and createdAt < @end_date")
comments['username'] = comments['author'].map(lambda a: a['login'])
comments['username_url'] = comments['author'].map(lambda a: a['url'])

In [129]:
commentors = (
    comments
    .groupby(['org', 'repo', 'username', 'authorAssociation'])
    .count()['author']
    .reset_index()
    .sort_values(['org', 'author'], ascending=False)
)

In [130]:
n_plot = 50
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='username',
        y='author',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    display(ch)

Chart({
  data:                org                repo             username authorAssociation  \
  0  binder-examples               bokeh            choldgraf            MEMBER   
  1  binder-examples               bokeh  mathematicalmichael              NONE   
  2  binder-examples          data-quilt              betatim            MEMBER   
  3  binder-examples          data-quilt          matheusmota              NONE   
  4  binder-examples          demo-julia            choldgraf            MEMBER   
  5  binder-examples  getting-data-xroot            choldgraf            MEMBER   
  6  binder-examples  getting-data-xroot      matthewfeickert       CONTRIBUTOR   
  7  binder-examples        jupyter-rise            choldgraf            MEMBER   
  8  binder-examples        jupyter-rise  mathematicalmichael              NONE   
  
     author  
  0       2  
  1       1  
  2       1  
  3       1  
  4       1  
  5       1  
  6       1  
  7       1  
  8       1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'author',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 commentors for binder-examples in the last 60 days',
  width: 1000
})

Chart({
  data:         org         repo        username authorAssociation  author
  20  ipython      ipython         Carreau            MEMBER      73
  13  ipython    ipykernel         hroncok       CONTRIBUTOR       7
  64  ipython    traitlets        rmorshea      COLLABORATOR       6
  15  ipython    ipykernel           minrk            MEMBER       5
  21  ipython      ipython  LucianaMarques       CONTRIBUTOR       5
  12  ipython    ipykernel      ccordoba12       CONTRIBUTOR       4
  35  ipython      ipython         efiring       CONTRIBUTOR       4
  50  ipython      ipython    sanketShetty              NONE       3
  60  ipython    traitlets     kevin-bates       CONTRIBUTOR       3
  9   ipython    ipykernel         Carreau            MEMBER       2
  11  ipython    ipykernel       blink1073            MEMBER       2
  18  ipython  ipyparallel           minrk            MEMBER       2
  26  ipython      ipython         blueyed       CONTRIBUTOR       2
  33  ipython      ipython          dlukes              NONE       2
  34  ipython      ipython     drunkwcodes              NONE       2
  37  ipython      ipython        fehiepsi              NONE       2
  42  ipython      ipython      jamesmyatt              NONE       2
  43  ipython      ipython        jfhbrook              NONE       2
  47  ipython      ipython          kmaork       CONTRIBUTOR       2
  49  ipython      ipython         mlucool              NONE       2
  53  ipython      ipython       tacaswell       CONTRIBUTOR       2
  55  ipython      ipython      thrasibule              NONE       2
  56  ipython      ipython         wenjuno              NONE       2
  58  ipython    traitlets           Jamim       CONTRIBUTOR       2
  63  ipython    traitlets          rgbkrk            MEMBER       2
  10  ipython    ipykernel         Chilipp       CONTRIBUTOR       1
  14  ipython    ipykernel      jamesmyatt              NONE       1
  16  ipython    ipykernel          noklam              NONE       1
  17  ipython  ipyparallel           Jamim       CONTRIBUTOR       1
  19  ipython      ipython    Benjamin-Lee              NONE       1
  22  ipython      ipython      OriolAbril       CONTRIBUTOR       1
  23  ipython      ipython        aakhmetz              NONE       1
  24  ipython      ipython            alok       CONTRIBUTOR       1
  25  ipython      ipython         anntzer       CONTRIBUTOR       1
  27  ipython      ipython  brandonwillard       CONTRIBUTOR       1
  28  ipython      ipython    brunobeltran              NONE       1
  29  ipython      ipython      ccordoba12       CONTRIBUTOR       1
  30  ipython      ipython         csheaff              NONE       1
  31  ipython      ipython          daharn       CONTRIBUTOR       1
  32  ipython      ipython         dickmao              NONE       1
  36  ipython      ipython  farisachugthai              NONE       1
  38  ipython      ipython       gdhameeja              NONE       1
  39  ipython      ipython        gpotter2       CONTRIBUTOR       1
  40  ipython      ipython         hroncok       CONTRIBUTOR       1
  41  ipython      ipython      ikizhvatov              NONE       1
  44  ipython      ipython       jmansilla              NONE       1
  45  ipython      ipython  joaomarcosgris              NONE       1
  46  ipython      ipython           kernc              NONE       1
  48  ipython      ipython           kmtac              NONE       1
  51  ipython      ipython        stonebig       CONTRIBUTOR       1,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      

Chart({
  data:          org                repo       username authorAssociation  author
  198  jupyter            nbgrader       jhamrick            MEMBER      98
  296  jupyter         repo2docker        betatim      COLLABORATOR      73
  274  jupyter            notebook      takluyver            MEMBER      69
  91   jupyter  enterprise_gateway    kevin-bates            MEMBER      62
  105  jupyter        jupyter-book      choldgraf            MEMBER      45
  166  jupyter           nbconvert          MSeal            MEMBER      40
  119  jupyter      jupyter-sphinx       akhmerov            MEMBER      36
  153  jupyter      jupyter_server        Zsailer            MEMBER      28
  318  jupyter         repo2docker      yuvipanda            MEMBER      26
  299  jupyter         repo2docker   consideRatio      COLLABORATOR      22
  319  jupyter           telemetry        Zsailer            MEMBER      20
  134  jupyter      jupyter_client  SylvainCorlay            MEMBER      19
  322  jupyter           telemetry      yuvipanda            MEMBER      17
  178  jupyter           nbconvert       t-makaro      COLLABORATOR      16
  202  jupyter            nbgrader        nthiery       CONTRIBUTOR      16
  206  jupyter            nbgrader        rkdarst       CONTRIBUTOR      16
  114  jupyter        jupyter-book    psychemedia              NONE      15
  283  jupyter           qtconsole     ccordoba12      COLLABORATOR      15
  152  jupyter      jupyter_server  SylvainCorlay            MEMBER      14
  214  jupyter            nbviewer       krinsman       CONTRIBUTOR      14
  320  jupyter           telemetry     jaipreet-s      COLLABORATOR      14
  87   jupyter  enterprise_gateway       echarles       CONTRIBUTOR      13
  133  jupyter      jupyter_client          MSeal            MEMBER      12
  159  jupyter      jupyter_server       rolweber      COLLABORATOR      12
  298  jupyter         repo2docker      choldgraf            MEMBER      12
  308  jupyter         repo2docker          minrk            MEMBER      12
  95   jupyter  enterprise_gateway       suryag10              NONE      11
  256  jupyter            notebook  joshuazeltser       CONTRIBUTOR      11
  272  jupyter            notebook     ssanderson       CONTRIBUTOR      10
  300  jupyter         repo2docker   davidanthoff       CONTRIBUTOR      10
  118  jupyter      jupyter-sphinx  SylvainCorlay            MEMBER       9
  150  jupyter        jupyter_core      takluyver            MEMBER       9
  188  jupyter            nbgrader        Sefriol              NONE       9
  205  jupyter            nbgrader    psychemedia              NONE       9
  263  jupyter            notebook       lresende            MEMBER       9
  93   jupyter  enterprise_gateway      michzimny       CONTRIBUTOR       8
  185  jupyter              nbdime        vidartf      COLLABORATOR       8
  273  jupyter            notebook       t-makaro       CONTRIBUTOR       8
  78   jupyter       docker-stacks        parente            MEMBER       7
  193  jupyter            nbgrader        dsblank            MEMBER       7
  194  jupyter            nbgrader     gertingold       CONTRIBUTOR       7
  88   jupyter  enterprise_gateway         esevan      COLLABORATOR       6
  122  jupyter      jupyter-sphinx   hugokerstens       CONTRIBUTOR       6
  145  jupyter      jupyter_client         rgbkrk            MEMBER       6
  216  jupyter            nbviewer        parente            MEMBER       6
  301  jupyter         repo2docker     davidrpugh       CONTRIBUTOR       6
  83   jupyter  enterprise_gateway       IMAM9AIS       CONTRIBUTOR       5
  101  jupyter             jupyter        parente            MEMBER       5
  103  jupyter        jupyter-book          BENR0              NONE       5
  106  jupyter        jupyter-book      cpjobling              NONE       5,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
     

Chart({
  data:                  org                repo         username authorAssociation  \
  346  jupyter-widgets          ipywidgets    SylvainCorlay            MEMBER   
  356  jupyter-widgets          ipywidgets       jasongrout            MEMBER   
  334  jupyter-widgets          ipyleaflet      martinRenou            MEMBER   
  370  jupyter-widgets          ipywidgets          vidartf            MEMBER   
  390  jupyter-widgets            tutorial          mwcraig       CONTRIBUTOR   
  383  jupyter-widgets           pythreejs          vidartf            MEMBER   
  354  jupyter-widgets          ipywidgets            hainm              NONE   
  337  jupyter-widgets          ipyleaflet         stonebig       CONTRIBUTOR   
  389  jupyter-widgets            tutorial      martinRenou            MEMBER   
  333  jupyter-widgets          ipyleaflet        kgiacobbi       CONTRIBUTOR   
  362  jupyter-widgets          ipywidgets          mwcraig       CONTRIBUTOR   
  345  jupyter-widgets          ipywidgets        Ricyteach       CONTRIBUTOR   
  378  jupyter-widgets           pythreejs         aliddell              NONE   
  332  jupyter-widgets          ipyleaflet            jtpio       CONTRIBUTOR   
  358  jupyter-widgets          ipywidgets          kafonek              NONE   
  359  jupyter-widgets          ipywidgets  maartenbreddels            MEMBER   
  365  jupyter-widgets          ipywidgets       rskabelund              NONE   
  388  jupyter-widgets            tutorial  maartenbreddels            MEMBER   
  351  jupyter-widgets          ipywidgets        choldgraf              NONE   
  366  jupyter-widgets          ipywidgets       ssanderson       CONTRIBUTOR   
  374  jupyter-widgets  jupyterlab-sidecar       doutriaux1              NONE   
  381  jupyter-widgets           pythreejs         stonebig              NONE   
  328  jupyter-widgets          ipyleaflet     VasavanThiru       CONTRIBUTOR   
  329  jupyter-widgets          ipyleaflet         chbrandt              NONE   
  330  jupyter-widgets          ipyleaflet         deeplook       CONTRIBUTOR   
  331  jupyter-widgets          ipyleaflet      dwr-psandhu              NONE   
  339  jupyter-widgets          ipywidgets  DanielAristidou              NONE   
  344  jupyter-widgets          ipywidgets        Liwellyen              NONE   
  349  jupyter-widgets          ipywidgets        beasteers              NONE   
  367  jupyter-widgets          ipywidgets         stonebig       CONTRIBUTOR   
  372  jupyter-widgets  jupyterlab-sidecar    SylvainCorlay            MEMBER   
  376  jupyter-widgets        midicontrols       jasongrout            MEMBER   
  391  jupyter-widgets            tutorial          vidartf            MEMBER   
  326  jupyter-widgets          ipyleaflet        Juanlu001       CONTRIBUTOR   
  327  jupyter-widgets          ipyleaflet    SylvainCorlay            MEMBER   
  335  jupyter-widgets          ipyleaflet         mpdwulit       CONTRIBUTOR   
  336  jupyter-widgets          ipyleaflet      paulathomas              NONE   
  338  jupyter-widgets          ipywidgets       DYSpider13              NONE   
  340  jupyter-widgets          ipywidgets         DevFabrx              NONE   
  341  jupyter-widgets          ipywidgets   IlyaKazakevich       CONTRIBUTOR   
  342  jupyter-widgets          ipywidgets        JohnIssue              NONE   
  343  jupyter-widgets          ipywidgets        Juanlu001       CONTRIBUTOR   
  347  jupyter-widgets          ipywidgets         akhmerov       CONTRIBUTOR   
  348  jupyter-widgets          ipywidgets   artur-trzesiok              NONE   
  350  jupyter-widgets          ipywidgets           bybyte              NONE   
  352  jupyter-widgets          ipywidgets      cornhundred       CONTRIBUTOR   
  353  jupyter-widgets          ipywidgets         dazzag24              NONE   
  355  jupyter-widgets          ipywidgets           ivanov       CONTRIBUTOR   
  357  jupyter-wi

Chart({
  data:             org                     repo         username authorAssociation  \
  590  jupyterhub  the-littlest-jupyterhub        yuvipanda            MEMBER   
  509  jupyterhub      mybinder.org-deploy          betatim            MEMBER   
  549  jupyterhub             team-compass          betatim            MEMBER   
  550  jupyterhub             team-compass        choldgraf            MEMBER   
  404  jupyterhub                binderhub          betatim            MEMBER   
  609  jupyterhub   zero-to-jupyterhub-k8s     consideRatio            MEMBER   
  529  jupyterhub              nbgitpuller        yuvipanda            MEMBER   
  487  jupyterhub               jupyterhub          ryogesh       CONTRIBUTOR   
  518  jupyterhub      mybinder.org-deploy      mael-le-gal            MEMBER   
  511  jupyterhub      mybinder.org-deploy        choldgraf            MEMBER   
  399  jupyterhub             batchspawner          rkdarst       CONTRIBUTOR   
  566  jupyterhub  the-littlest-jupyterhub   GeorgianaElena      COLLABORATOR   
  560  jupyterhub             team-compass        sgibson91            MEMBER   
  406  jupyterhub                binderhub        choldgraf            MEMBER   
  558  jupyterhub             team-compass            minrk            MEMBER   
  523  jupyterhub              nbgitpuller        choldgraf            MEMBER   
  596  jupyterhub            traefik-proxy            minrk            MEMBER   
  407  jupyterhub                binderhub     consideRatio            MEMBER   
  478  jupyterhub               jupyterhub            minrk            MEMBER   
  521  jupyterhub      mybinder.org-deploy        yuvipanda            MEMBER   
  526  jupyterhub              nbgitpuller      parmentelat       CONTRIBUTOR   
  562  jupyterhub             team-compass        yuvipanda            MEMBER   
  418  jupyterhub                binderhub        sgibson91            MEMBER   
  513  jupyterhub      mybinder.org-deploy           henchc            MEMBER   
  594  jupyterhub            traefik-proxy   GeorgianaElena      COLLABORATOR   
  617  jupyterhub   zero-to-jupyterhub-k8s           manics       CONTRIBUTOR   
  443  jupyterhub               jupyterhub   GeorgianaElena       CONTRIBUTOR   
  447  jupyterhub               jupyterhub  RandyBetancourt              NONE   
  490  jupyterhub               jupyterhub        vilhelmen       CONTRIBUTOR   
  497  jupyterhub              kubespawner     consideRatio            MEMBER   
  615  jupyterhub   zero-to-jupyterhub-k8s         joshbode       CONTRIBUTOR   
  397  jupyterhub             batchspawner       mbmilligan            MEMBER   
  492  jupyterhub               jupyterhub        yuvipanda            MEMBER   
  519  jupyterhub      mybinder.org-deploy            minrk            MEMBER   
  538  jupyterhub           oauthenticator            minrk            MEMBER   
  547  jupyterhub             team-compass          Zsailer            MEMBER   
  557  jupyterhub             team-compass      mael-le-gal            MEMBER   
  563  jupyterhub  the-littlest-jupyterhub      CyborgDroid              NONE   
  415  jupyterhub                binderhub         memeplex              NONE   
  459  jupyterhub               jupyterhub      brayanrodpr              NONE   
  499  jupyterhub              kubespawner           gsemet       CONTRIBUTOR   
  561  jupyterhub             team-compass         willingc            MEMBER   
  616  jupyterhub   zero-to-jupyterhub-k8s        lakshaykc              NONE   
  409  jupyterhub                binderhub          jhamman       CONTRIBUTOR   
  416  jupyterhub                binderhub            minrk            MEMBER   
  420  jupyterhub                binderhub        yuvipanda            MEMBER   
  438  jupyterhub     jupyter-server-proxy         quasiben       CONTRIBUTOR   
  466  jupyterhub               jupyterhub          eylenth              NONE   
  472  jupyterhub

Chart({
  data:             org                      repo             username  \
  724  jupyterlab                jupyterlab           jasongrout   
  675  jupyterlab                jupyterlab            blink1073   
  726  jupyterlab                jupyterlab  jupyterlab-dev-mode   
  718  jupyterlab                jupyterlab           ian-r-rose   
  780  jupyterlab                jupyterlab              vidartf   
  763  jupyterlab                jupyterlab       saulshanabrook   
  770  jupyterlab                jupyterlab           telamonian   
  701  jupyterlab                jupyterlab            ellisonbg   
  660  jupyterlab                jupyterlab               afshin   
  803  jupyterlab            jupyterlab-git           jaipreet-s   
  639  jupyterlab                jupyterlab               BoPeng   
  771  jupyterlab                jupyterlab            tgeorgeux   
  666  jupyterlab                jupyterlab             aschlaep   
  789  jupyterlab  jupyterlab-data-explorer       saulshanabrook   
  696  jupyterlab                jupyterlab             domoritz   
  775  jupyterlab                jupyterlab              tslaton   
  722  jupyterlab                jupyterlab              jakevdp   
  725  jupyterlab                jupyterlab                jtpio   
  734  jupyterlab                jupyterlab           krassowski   
  657  jupyterlab                jupyterlab              Zsailer   
  699  jupyterlab                jupyterlab         duarteocarmo   
  782  jupyterlab                jupyterlab         vtrokhymenko   
  794  jupyterlab           jupyterlab-demo           ian-r-rose   
  795  jupyterlab           jupyterlab-demo           jasongrout   
  712  jupyterlab                jupyterlab              gnestor   
  719  jupyterlab                jupyterlab              ibdafna   
  747  jupyterlab                jupyterlab           michaelaye   
  768  jupyterlab                jupyterlab             stonebig   
  802  jupyterlab            jupyterlab-git           fcollonval   
  811  jupyterlab            jupyterlab-git       saulshanabrook   
  773  jupyterlab                jupyterlab            timkpaine   
  823  jupyterlab         jupyterlab_server            blink1073   
  644  jupyterlab                jupyterlab           IgorDouven   
  673  jupyterlab                jupyterlab            bionicles   
  703  jupyterlab                jupyterlab           fcollonval   
  720  jupyterlab                jupyterlab               ivanov   
  752  jupyterlab                jupyterlab           nickeubank   
  807  jupyterlab            jupyterlab-git  mathematicalmichael   
  674  jupyterlab                jupyterlab              bjtho08   
  691  jupyterlab                jupyterlab         davidanthoff   
  700  jupyterlab                jupyterlab             echarles   
  707  jupyterlab                jupyterlab         flying-sheep   
  721  jupyterlab                jupyterlab           jaipreet-s   
  723  jupyterlab                jupyterlab            jakirkham   
  786  jupyterlab                jupyterlab            yuvipanda   
  805  jupyterlab            jupyterlab-git           luispuerto   
  668  jupyterlab                jupyterlab            athornton   
  672  jupyterlab                jupyterlab              betatim   
  686  jupyterlab                jupyterlab           daejong123   
  690  jupyterlab                jupyterlab     david-waterworth   
  
      authorAssociation  author  
  724            MEMBER     421  
  675            MEMBER     237  
  726              NONE     147  
  718            MEMBER     126  
  780            MEMBER     126  
  763            MEMBER      47  
  770            MEMBER      47  
  701            MEMBER      44  
  660            MEMBER      43  
  803      COLLABORATOR      22  
  639            MEMBER      21  
  771            MEMBER      20  
  666            MEMBER      17  
  789            MEMBER      17  
  696

Chart({
  data:          org            repo             username authorAssociation  author
  876  nteract        hydrogen       wadethestealth            MEMBER     137
  856  nteract        hydrogen             aviatesk            MEMBER      65
  841  nteract       bookstore               mpacer            MEMBER      51
  840  nteract       bookstore              codecov              NONE      40
  845  nteract       bookstore             willingc            MEMBER      36
  848  nteract        hydrogen           BenRussert            MEMBER      36
  911  nteract       papermill                MSeal            MEMBER      34
  888  nteract         nteract         captainsafia            MEMBER      18
  922  nteract       papermill               rgbkrk            MEMBER      18
  901  nteract         nteract               rgbkrk            MEMBER      15
  844  nteract       bookstore                 todo              NONE      13
  904  nteract         nteract                stale              NONE      13
  905  nteract         nteract       wadethestealth            MEMBER      11
  881  nteract         nteract           BenRussert            MEMBER      10
  921  nteract       papermill               mwouts            MEMBER      10
  925  nteract       scrapbook                MSeal            MEMBER      10
  908  nteract      nteract.io                  now              NONE       9
  862  nteract        hydrogen           kiwi0fruit       CONTRIBUTOR       8
  917  nteract       papermill         duarteocarmo            MEMBER       7
  859  nteract        hydrogen             hoishing            MEMBER       6
  868  nteract        hydrogen            pepicello              NONE       6
  874  nteract        hydrogen             stevengj              NONE       6
  885  nteract         nteract         ShalokShalom              NONE       6
  898  nteract         nteract      nicolaskruchten              NONE       5
  924  nteract       papermill             willingc            MEMBER       5
  931  nteract        semiotic               emeeks            MEMBER       5
  937  nteract            vdom             rmorshea            MEMBER       5
  867  nteract        hydrogen         optionsraghu              NONE       4
  872  nteract        hydrogen             shr00mie              NONE       4
  873  nteract        hydrogen      sschuldenzucker              NONE       4
  906  nteract         nteract             willingc            MEMBER       4
  912  nteract       papermill          RoelBrouwer       CONTRIBUTOR       4
  913  nteract       papermill  avisrivastava254084              NONE       4
  927  nteract       scrapbook           matthiasdv            MEMBER       4
  839  nteract       bookstore                MSeal            MEMBER       3
  842  nteract       bookstore               rgbkrk            MEMBER       3
  860  nteract        hydrogen             isajacas              NONE       3
  861  nteract        hydrogen          jesusloplar              NONE       3
  865  nteract        hydrogen             n-riesco            MEMBER       3
  880  nteract  minimal-plotly               rgbkrk            MEMBER       3
  909  nteract       papermill               0anton              NONE       3
  926  nteract       scrapbook              codecov              NONE       3
  935  nteract            vdom              gnestor            MEMBER       3
  850  nteract        hydrogen            Mikeymice              NONE       2
  852  nteract        hydrogen                Tbosa              NONE       2
  854  nteract        hydrogen              ald0405              NONE       2
  858  nteract        hydrogen                goi42              NONE       2
  863  nteract        hydrogen           kylebarron            MEMBER       2
  869  nteract        hydrogen             renovate       CONTRIBUTOR       2
  875  nteract        hydrogen                 todo              NONE       2,
  encod

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [131]:
first_responders = created.copy().reset_index(drop=True)
for ii, issue in first_responders.iterrows():
    comments = issue.get('comments', [])['edges']
    if len(comments) == 0:
        continue
    i_comments = pd.DataFrame([ii['node'] for ii in comments]).sort_values('createdAt')
    first_comment = i_comments.iloc[0]
    first_responders.loc[ii, 'first_responder'] = first_comment['author']['login']
    first_responders.loc[ii, 'first_responder_association'] = first_comment['authorAssociation']

In [132]:
first_responders = (
    first_responders
    .groupby(['org', 'repo', 'first_responder', 'first_responder_association'])
    .count()
    .rename(columns={'author': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [133]:
n_plot = 50
for ii, (iorg, idata) in enumerate(first_responders.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='first_responder',
        y='count',
        color=alt.Color('first_responder_association', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    display(ch)

Chart({
  data:                org          repo first_responder first_responder_association  \
  0  binder-examples    demo-julia       choldgraf                      MEMBER   
  1  binder-examples  jupyter-rise       choldgraf                      MEMBER   
  
     count  
  0      1  
  1      1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'first_responder_association',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'first_responder',
      type: 'nominal'
    }),
    y: Y({
      field: 'count',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 first responders for binder-examples in the last 60 days',
  width: 1000
})

Chart({
  data:         org              repo first_responder first_responder_association  \
  4   ipython           ipython         Carreau                      MEMBER   
  8   ipython           ipython     drunkwcodes                        NONE   
  2   ipython         ipykernel      ccordoba12                 CONTRIBUTOR   
  3   ipython           ipython    Benjamin-Lee                        NONE   
  5   ipython           ipython  LucianaMarques                 CONTRIBUTOR   
  6   ipython           ipython         dickmao                        NONE   
  7   ipython           ipython          dlukes                        NONE   
  9   ipython           ipython       gdhameeja                        NONE   
  10  ipython           ipython        jfhbrook                        NONE   
  11  ipython           ipython  joaomarcosgris                        NONE   
  12  ipython           ipython         tbeadle                        NONE   
  13  ipython  ipython-in-depth      jasongrout                      MEMBER   
  
      count  
  4      13  
  8       2  
  2       1  
  3       1  
  5       1  
  6       1  
  7       1  
  9       1  
  10      1  
  11      1  
  12      1  
  13      1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'first_responder_association',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'first_responder',
      type: 'nominal'
    }),
    y: Y({
      field: 'count',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 first responders for ipython in the last 60 days',
  width: 1000
})

Chart({
  data:         org                repo first_responder first_responder_association  \
  22  jupyter        jupyter-book       choldgraf                      MEMBER   
  71  jupyter         repo2docker         betatim                COLLABORATOR   
  46  jupyter            nbgrader        jhamrick                      MEMBER   
  38  jupyter           nbconvert           MSeal                      MEMBER   
  20  jupyter  enterprise_gateway     kevin-bates                      MEMBER   
  18  jupyter       docker-stacks         parente                      MEMBER   
  23  jupyter        jupyter-book         emdupre                COLLABORATOR   
  39  jupyter           nbconvert        t-makaro                COLLABORATOR   
  42  jupyter              nbdime         vidartf                COLLABORATOR   
  50  jupyter            nbviewer        krinsman                 CONTRIBUTOR   
  35  jupyter      jupyter_server        rolweber                COLLABORATOR   
  43  jupyter            nbgrader         Sefriol                        NONE   
  48  jupyter            nbgrader       perllaghu                        NONE   
  63  jupyter            notebook        jyu78749                        NONE   
  64  jupyter            notebook     kevin-bates                      MEMBER   
  74  jupyter         repo2docker          manics                        NONE   
  14  jupyter       accessibility          fperez                      MEMBER   
  15  jupyter       docker-stacks       daviddoji                        NONE   
  16  jupyter       docker-stacks   ejgutierrez74                        NONE   
  17  jupyter       docker-stacks    miramar-labs                        NONE   
  19  jupyter       docker-stacks        richtong                        NONE   
  21  jupyter             jupyter       choldgraf                      MEMBER   
  24  jupyter        jupyter-book     psychemedia                        NONE   
  25  jupyter        jupyter-book     thammegowda                 CONTRIBUTOR   
  26  jupyter      jupyter-sphinx    hugokerstens                 CONTRIBUTOR   
  27  jupyter      jupyter_client         dsblank                      MEMBER   
  28  jupyter      jupyter_client         jits023                        NONE   
  29  jupyter      jupyter_client     kevin-bates                      MEMBER   
  30  jupyter      jupyter_client         vidartf                        NONE   
  31  jupyter      jupyter_server   SylvainCorlay                      MEMBER   
  32  jupyter      jupyter_server         Zsailer                      MEMBER   
  33  jupyter      jupyter_server       blink1073                      MEMBER   
  34  jupyter      jupyter_server        lresende                      MEMBER   
  36  jupyter      kernel_gateway     kevin-bates                      MEMBER   
  37  jupyter               nb2kg     kevin-bates                      MEMBER   
  40  jupyter              nbdime       bavaria95                        NONE   
  41  jupyter              nbdime      jaipreet-s                        NONE   
  44  jupyter            nbgrader       Yenthe666                        NONE   
  45  jupyter            nbgrader         dsblank                      MEMBER   
  47  jupyter            nbgrader         ocefpaf                        NONE   
  49  jupyter            nbgrader     psychemedia                        NONE   
  51  jupyter            notebook    Elizaveta239                        NONE   
  52  jupyter            notebook          Iorent                        NONE   
  53  jupyter            notebook   WilliamTjiong                        NONE   
  54  jupyter            notebook         YuGlede                        NONE   
  55  jupyter            notebook    autodataming                        NONE   
  56  jupyter            notebook       blink1073                      MEMBER   
  57  jupyter            notebook        coldJune                        NONE   
  58  jupyter    

Chart({
  data:                 org                repo first_responder  \
  83  jupyter-widgets          ipywidgets   SylvainCorlay   
  93  jupyter-widgets          ipywidgets         vidartf   
  96  jupyter-widgets           pythreejs         vidartf   
  84  jupyter-widgets          ipywidgets       beasteers   
  88  jupyter-widgets          ipywidgets      jasongrout   
  80  jupyter-widgets          ipyleaflet     dwr-psandhu   
  81  jupyter-widgets          ipyleaflet     paulathomas   
  82  jupyter-widgets          ipywidgets       JohnIssue   
  85  jupyter-widgets          ipywidgets          bybyte   
  86  jupyter-widgets          ipywidgets       choldgraf   
  87  jupyter-widgets          ipywidgets           hainm   
  89  jupyter-widgets          ipywidgets     jquacinella   
  90  jupyter-widgets          ipywidgets         kafonek   
  91  jupyter-widgets          ipywidgets          piyueh   
  92  jupyter-widgets          ipywidgets      rskabelund   
  94  jupyter-widgets          ipywidgets     zxzhaixiang   
  95  jupyter-widgets  jupyterlab-sidecar      doutriaux1   
  97  jupyter-widgets            tutorial         vidartf   
  
     first_responder_association  count  
  83                      MEMBER      5  
  93                      MEMBER      4  
  96                      MEMBER      4  
  84                        NONE      2  
  88                      MEMBER      2  
  80                        NONE      1  
  81                        NONE      1  
  82                        NONE      1  
  85                        NONE      1  
  86                        NONE      1  
  87                        NONE      1  
  89                        NONE      1  
  90                        NONE      1  
  91                        NONE      1  
  92                        NONE      1  
  94                        NONE      1  
  95                        NONE      1  
  97                      MEMBER      1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'first_responder_association',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'first_responder',
      type: 'nominal'
    }),
    y: Y({
      field: 'count',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 first responders for jupyter-widgets in the last 60 days',
  width: 1000
})

Chart({
  data:             org                     repo     first_responder  \
  101  jupyterhub                binderhub             betatim   
  127  jupyterhub      mybinder.org-deploy             betatim   
  138  jupyterhub             team-compass             betatim   
  121  jupyterhub               jupyterhub             ryogesh   
  149  jupyterhub   zero-to-jupyterhub-k8s        consideRatio   
  141  jupyterhub  the-littlest-jupyterhub      GeorgianaElena   
  131  jupyterhub              nbgitpuller           yuvipanda   
  145  jupyterhub  the-littlest-jupyterhub           yuvipanda   
  139  jupyterhub             team-compass           choldgraf   
  140  jupyterhub             team-compass               minrk   
  148  jupyterhub              wrapspawner             rkdarst   
  99   jupyterhub             batchspawner             rkdarst   
  104  jupyterhub                binderhub           sgibson91   
  115  jupyterhub               jupyterhub            katsar0v   
  120  jupyterhub               jupyterhub          ryanlovett   
  124  jupyterhub              kubespawner        consideRatio   
  143  jupyterhub  the-littlest-jupyterhub     lachlancampbell   
  151  jupyterhub   zero-to-jupyterhub-k8s              manics   
  152  jupyterhub   zero-to-jupyterhub-k8s   sampathkethineedi   
  98   jupyterhub             batchspawner            rcthomas   
  100  jupyterhub                binderhub               arnim   
  102  jupyterhub                binderhub        consideRatio   
  103  jupyterhub                binderhub              manics   
  105  jupyterhub            dockerspawner            Redhappy   
  106  jupyterhub            dockerspawner    ZelphirKaltstahl   
  107  jupyterhub            dockerspawner  baijingetangzichen   
  108  jupyterhub            dockerspawner           timhughes   
  109  jupyterhub            dockerspawner         yakutovicha   
  110  jupyterhub   jupyter-rsession-proxy              xhochy   
  111  jupyterhub     jupyter-server-proxy     juliusvonkohout   
  112  jupyterhub               jupyterhub      GeorgianaElena   
  113  jupyterhub               jupyterhub         UCSBGauchos   
  114  jupyterhub               jupyterhub           casperdcl   
  116  jupyterhub               jupyterhub            krinsman   
  117  jupyterhub               jupyterhub              manics   
  118  jupyterhub               jupyterhub         ramonberger   
  119  jupyterhub               jupyterhub          rochaporto   
  122  jupyterhub               jupyterhub           vilhelmen   
  123  jupyterhub               jupyterhub            wqh17101   
  125  jupyterhub              kubespawner              gsemet   
  126  jupyterhub              kubespawner      julienchastang   
  128  jupyterhub      mybinder.org-deploy           choldgraf   
  129  jupyterhub              nbgitpuller      albertmichaelj   
  130  jupyterhub              nbgitpuller            mpilosov   
  132  jupyterhub           oauthenticator  hedgehog-millenium   
  133  jupyterhub           oauthenticator            inviscid   
  134  jupyterhub           oauthenticator               minrk   
  135  jupyterhub           oauthenticator           yuvipanda   
  136  jupyterhub           systemdspawner            rstuckey   
  137  jupyterhub             team-compass             Zsailer   
  
      first_responder_association  count  
  101                      MEMBER      9  
  127                      MEMBER      8  
  138                      MEMBER      8  
  121                 CONTRIBUTOR      7  
  149                      MEMBER      6  
  141                COLLABORATOR      5  
  131                      MEMBER      4  
  145                      MEMBER      4  
  139                      MEMBER      3  
  140                      MEMBER      3  
  148                        NONE      3  
  99                  CONTRIBUTOR      2  
  104                      MEMBER      2  
  115          

Chart({
  data:             org                       repo  first_responder  \
  180  jupyterlab                 jupyterlab       jasongrout   
  168  jupyterlab                 jupyterlab        blink1073   
  196  jupyterlab                 jupyterlab          vidartf   
  184  jupyterlab                 jupyterlab       krassowski   
  199  jupyterlab   jupyterlab-data-explorer   saulshanabrook   
  177  jupyterlab                 jupyterlab       ian-r-rose   
  158  jupyterlab                 jupyterlab           BoPeng   
  159  jupyterlab                 jupyterlab       IgorDouven   
  165  jupyterlab                 jupyterlab           afshin   
  172  jupyterlab                 jupyterlab         echarles   
  173  jupyterlab                 jupyterlab        ellisonbg   
  181  jupyterlab                 jupyterlab            jtpio   
  191  jupyterlab                 jupyterlab       telamonian   
  194  jupyterlab                 jupyterlab          tslaton   
  155  jupyterlab  extension-cookiecutter-ts       jasongrout   
  156  jupyterlab          jupyter-renderers        jonmmease   
  157  jupyterlab                 jupyterlab      AlokTakshak   
  160  jupyterlab                 jupyterlab        Koushik95   
  161  jupyterlab                 jupyterlab         SamKacer   
  162  jupyterlab                 jupyterlab         SgtPooki   
  163  jupyterlab                 jupyterlab  SultanOrazbayev   
  164  jupyterlab                 jupyterlab            Zlass   
  166  jupyterlab                 jupyterlab       alesgenova   
  167  jupyterlab                 jupyterlab         aschlaep   
  169  jupyterlab                 jupyterlab      claraabello   
  170  jupyterlab                 jupyterlab       daejong123   
  171  jupyterlab                 jupyterlab       doutriaux1   
  174  jupyterlab                 jupyterlab     felixdollack   
  175  jupyterlab                 jupyterlab     flying-sheep   
  176  jupyterlab                 jupyterlab         groverj3   
  178  jupyterlab                 jupyterlab          ibdafna   
  179  jupyterlab                 jupyterlab       jaipreet-s   
  182  jupyterlab                 jupyterlab        kanekalla   
  183  jupyterlab                 jupyterlab        kapilkd13   
  185  jupyterlab                 jupyterlab    markellekelly   
  186  jupyterlab                 jupyterlab        marwahaha   
  187  jupyterlab                 jupyterlab     michael-burt   
  188  jupyterlab                 jupyterlab   mouradmourafiq   
  189  jupyterlab                 jupyterlab   saulshanabrook   
  190  jupyterlab                 jupyterlab         stonebig   
  192  jupyterlab                 jupyterlab        tgeorgeux   
  193  jupyterlab                 jupyterlab        timkpaine   
  195  jupyterlab                 jupyterlab            van-8   
  197  jupyterlab        jupyterlab-celltags            e01n0   
  198  jupyterlab      jupyterlab-commenting           hoo761   
  200  jupyterlab             jupyterlab-git      DiegoPiloni   
  201  jupyterlab             jupyterlab-git      dhirschfeld   
  202  jupyterlab             jupyterlab-git       fcollonval   
  203  jupyterlab             jupyterlab-git       jaipreet-s   
  204  jupyterlab    jupyterlab-google-drive       ian-r-rose   
  
      first_responder_association  count  
  180                      MEMBER     26  
  168                      MEMBER     10  
  196                      MEMBER      9  
  184                      MEMBER      4  
  199                      MEMBER      4  
  177                      MEMBER      3  
  158                      MEMBER      2  
  159                        NONE      2  
  165                      MEMBER      2  
  172                      MEMBER      2  
  173                      MEMBER      2  
  181                      MEMBER      2  
  191                      MEMBER      2  
  194                 CONTRIBUTOR      2  
  155                  

Chart({
  data:          org            repo  first_responder first_responder_association  \
  228  nteract       papermill            MSeal                      MEMBER   
  220  nteract         nteract     captainsafia                      MEMBER   
  216  nteract        hydrogen   wadethestealth                      MEMBER   
  209  nteract       bookstore           mpacer                      MEMBER   
  225  nteract         nteract   wadethestealth                      MEMBER   
  230  nteract       scrapbook            MSeal                      MEMBER   
  211  nteract       bookstore         willingc                      MEMBER   
  214  nteract        hydrogen         aviatesk                      MEMBER   
  210  nteract       bookstore           rgbkrk                      MEMBER   
  212  nteract              dx       dylanross3                      MEMBER   
  213  nteract        hydrogen       BenRussert                      MEMBER   
  215  nteract        hydrogen         n-riesco                      MEMBER   
  217  nteract  minimal-plotly  nicolaskruchten                        NONE   
  218  nteract         nteract       BenRussert                      MEMBER   
  219  nteract         nteract          bnbarak                      MEMBER   
  221  nteract         nteract             drbh                        NONE   
  222  nteract         nteract      hydrosquall                      MEMBER   
  223  nteract         nteract          mattbit                        NONE   
  224  nteract         nteract           rgbkrk                      MEMBER   
  226  nteract         nteract         willingc                      MEMBER   
  227  nteract         nteract       yogevyuval                        NONE   
  229  nteract       papermill         willingc                      MEMBER   
  231  nteract        semiotic      micahstubbs                 CONTRIBUTOR   
  232  nteract        semiotic            niwsa                        NONE   
  233  nteract            vdom         rmorshea                      MEMBER   
  
       count  
  228      8  
  220      5  
  216      4  
  209      3  
  225      3  
  230      3  
  211      2  
  214      2  
  210      1  
  212      1  
  213      1  
  215      1  
  217      1  
  218      1  
  219      1  
  221      1  
  222      1  
  223      1  
  224      1  
  226      1  
  227      1  
  229      1  
  231      1  
  232      1  
  233      1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'first_responder_association',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'first_responder',
      type: 'nominal'
    }),
    y: Y({
      field: 'count',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 first responders for nteract in the last 60 days',
  width: 1000
})

In [134]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>